In [1]:
import sys
sys.path.append("../")

In [2]:
from data_tools.containers import StocksData, StocksTarget
from data_tools.model_preprocessors import SequenceModelPreprocessor, StocksDataModelPreprocessor
from targets.target_function import ClosePriceTargetFunction
from models import LSTMRegressionModel
from features import IndicatorsFeaturesList

from sklearn.metrics import mean_absolute_percentage_error

c:\ProgramData\Anaconda3\envs\trading-algorithms\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
stocks_data = StocksData("../data/market_data/AAPL.csv")
stocks_data = IndicatorsFeaturesList().transform(stocks_data)

In [5]:
stocks_target = StocksTarget.from_target_function(stocks_data, ClosePriceTargetFunction(bars_count=3))

In [6]:
train_data, val_data, train_target, val_target = StocksDataModelPreprocessor.split(stocks_data, stocks_target, threshold="2022-11-12")

preprocessor = SequenceModelPreprocessor(sequence_lenght=48)

X_train, y_train = preprocessor.preprocess(train_data, train_target)
X_val, y_val = preprocessor.preprocess(val_data, val_target)

In [6]:
model = LSTMRegressionModel(stocks_data.num_features())

model.fit(X_train, y_train, max_epochs=200)